# Trabalho de Mineração de Dados
### Por: Davi de Castro e João Pedro Righi

* Tentamos requisitar e usar o json diretamente mas havia um erro nele.

* Havia um erro no json onde faltava uma vírgula em determinado item, 
por ser apenas um caso isolado escolhemos mudar manualmente.

* Caso fosse um erro recorrente poderíamos utilizar uma lógica de verificação e inserção de vírgulas faltantes

In [7]:
#  Substituir "\u00e3" por "a"

import json

def substituir_unicode(input_file, output_file):
    # Carrega o conteúdo do arquivo JSON original
    with open(input_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # Converte o JSON para uma string
    json_str = json.dumps(data)
    
    # Substitui "\u00e3" por "a"
    json_str_modificado = json_str.replace("\u00e3", "a")
    
    # Converte a string de volta para um objeto JSON
    data_modificado = json.loads(json_str_modificado)
    
    # Salva o resultado em um novo arquivo
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(data_modificado, file, ensure_ascii=False, indent=4)

# Caminho para os arquivos de entrada e saída
arquivo_entrada = 'padaria_trab.json'
arquivo_saida = 'saida.json'

# Chama a função para substituir o unicode
substituir_unicode(arquivo_entrada, arquivo_saida)

print(f"Substituição concluída. O resultado foi salvo em {arquivo_saida}.")


Substituição concluída. O resultado foi salvo em saida.json.


* Havia um erro de formatação que foi resolvido substituindo "\u00e3" por "a". 

* Agora vamos remover a chave "compra", uma vez que o índice do objeto já será o bastante para diferenciá-lo dos demais.

In [8]:
import json
import unicodedata

def remover_acentos(texto):
    """Remove acentos de um texto e converte para minúsculas."""
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto.lower())
        if unicodedata.category(c) != 'Mn'
    )

def processar_json(input_file, output_file):
    """Lê um JSON, remove a chave 'compra', e processa os produtos removendo acentos e convertendo para minúsculas."""
    # Carregar os dados do arquivo JSON
    with open(input_file, 'r') as file:
        compras = json.load(file)

    # Processar cada compra
    for compra in compras:
        # Remover a chave 'compra'
        if 'compra' in compra:
            del compra['compra']
        
        # Remover acentos e converter para minúsculas os nomes dos produtos
        compra['produtos'] = [remover_acentos(produto) for produto in compra['produtos']]

    # Salvar os dados modificados em um novo arquivo JSON
    with open(output_file, 'w') as file:
        json.dump(compras, file, indent=4)

    print(f"A chave 'compra' foi removida e os acentos foram removidos. Dados salvos em '{output_file}'.")

# Chamar a função com os nomes dos arquivos
processar_json('saida.json', 'saida_transformada.json')


A chave 'compra' foi removida e os acentos foram removidos. Dados salvos em 'saida_transformada.json'.


* Agora temos um arquivo json otimizado para a etapa de mineração de dados.

In [15]:
import json
import pandas as pd
import unicodedata
from mlxtend.frequent_patterns import apriori, association_rules

def remover_acentos(texto):
    """Remove acentos de um texto e converte para minúsculas."""
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto.lower())
        if unicodedata.category(c) != 'Mn'
    )

def carregar_compras(input_file):
    """Lê o arquivo JSON e retorna uma lista de listas com os produtos."""
    try:
        with open(input_file, 'r') as file:
            compras = json.load(file)
        
        # Processar os produtos, removendo acentos e convertendo para minúsculas
        return [[remover_acentos(produto) for produto in compra.get('produtos', [])] for compra in compras]
    except (json.JSONDecodeError, FileNotFoundError) as e:
        print(f"Erro ao carregar o arquivo JSON: {e}")
        return []

def gerar_regras_associacao(input_file, min_suporte=0.05, min_confianca=0.2):
    """Gera regras de associação a partir de um conjunto de dados de compras."""
    compras = carregar_compras(input_file)

    if not compras:
        print("Nenhuma compra foi carregada. Verifique o arquivo de entrada.")
        return None, None, None

    # Criar DataFrame com transações codificadas como hot encoding
    all_products = set(produto for compra in compras for produto in compra)
    df = pd.DataFrame([{produto: (produto in compra) for produto in all_products} for compra in compras])

    # Aplicar o algoritmo Apriori para encontrar itens frequentes
    frequent_itemsets = apriori(df, min_support=min_suporte, use_colnames=True)

    # Gerar regras de associação
    regras = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confianca)

    # Encontrar as 5 principais regras baseadas na confiança
    top_5_regras = regras.nlargest(5, 'confidence')

    # Encontrar a regra mais influente 1 para 1 (i.e., um produto implicando diretamente em outro)
    regra_mais_influente = regras[(regras['antecedents'].apply(len) == 1) & (regras['consequents'].apply(len) == 1)].nlargest(1, 'confidence')

    # Encontrar regras que implicam em comprar "doce"
    regras_para_doce = regras[regras['consequents'].apply(lambda x: any('doce' in item for item in x))]

    return top_5_regras, regra_mais_influente, regras_para_doce

# Chamar a função para gerar as regras
top_5_regras, regra_mais_influente, regras_para_doce = gerar_regras_associacao('saida_transformada.json', min_suporte=0.02, min_confianca=0.1)

def salvar_resultados_em_arquivo(output_file, top_5_regras, regra_mais_influente, regras_para_doce):
    with open(output_file, 'w') as file:
        if top_5_regras is not None:
            file.write("Top 5 Regras de Associação:\n\n")
            for index, row in top_5_regras.iterrows():
                file.write(f"Regra {index + 1}: Se {set(row['antecedents'])} então {set(row['consequents'])}\n")
                file.write(f"  - Suporte: {row['support']:.4f}\n")
                file.write(f"  - Confiança: {row['confidence']:.4f}\n")
                file.write(f"  - Lift: {row['lift']:.4f}\n\n")

        file.write("\nRegra Mais Influente 1 para 1:\n\n")
        if regra_mais_influente is not None and not regra_mais_influente.empty:
            row = regra_mais_influente.iloc[0]
            file.write(f"Se {set(row['antecedents'])} então {set(row['consequents'])}\n")
            file.write(f"  - Suporte: {row['support']:.4f}\n")
            file.write(f"  - Confiança: {row['confidence']:.4f}\n")
            file.write(f"  - Lift: {row['lift']:.4f}\n\n")
        else:
            file.write("Nenhuma regra 1 para 1 encontrada.\n\n")

        file.write("\nRegras que Implicam na Compra de 'Doce':\n\n")
        if regras_para_doce is not None and not regras_para_doce.empty:
            for index, row in regras_para_doce.iterrows():
                file.write(f"Regra {index + 1}: Se {set(row['antecedents'])} então {set(row['consequents'])}\n")
                file.write(f"  - Suporte: {row['support']:.4f}\n")
                file.write(f"  - Confiança: {row['confidence']:.4f}\n")
                file.write(f"  - Lift: {row['lift']:.4f}\n\n")
        else:
            file.write("Nenhuma regra encontrada que implique na compra de 'doce'.\n")

    print(f"Resultados salvos em '{output_file}'.")

# Chamar a função para salvar os resultados em um arquivo
salvar_resultados_em_arquivo('resultados_associacao.txt', top_5_regras, regra_mais_influente, regras_para_doce)



Resultados salvos em 'resultados_associacao.txt'.
